In [2]:
!pip install torch

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/d6/a8/43e5033f9b2f727c158456e0720f870030ad3685c46f41ca3ca901b54922/torch-2.1.1-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/192.3 MB ? eta -:--:--
   ---------------------------------------- 0.3/192.3 MB 7.9 MB/s eta 0:00:25
   ---------------------------------------- 0.6/192.3 MB 7.7 MB/s eta 0:00:25
   ---------------------------------------- 1.5/192.3 MB 10.9 MB/s eta 0:00:18
   ---------------------------------------- 2.3/192.3 MB 12.0 MB/s eta 0:00:16
    --------------------------------------- 2.7/192.3 MB 11.6 MB/s eta 0:00:17
    --------------------------------------- 3.6/192.3 MB 12.1 MB/s eta 0:00:16
    --------------------------------------- 4.4/192.3 MB 12.7 MB/s eta 0:00:15
   - -------------------------------------- 5.2/192.3 MB 13.7 MB/s eta 0:00:14
   - -------------------------------------- 6.0/192.3 MB 14.1 MB/s eta 0:00:14
   - -----------

In [6]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split


def load_data(file_path):
    data = pd.read_csv(file_path, header=None)
    X = data.iloc[:, :-1].values
    y = data.iloc[:, -1].values.reshape(-1, 1)
    return X, y

# Define the neural network class
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, activation):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        self.activation = activation

        # Initialize weights based on the activation function
        if activation == 'tanh':
            nn.init.xavier_uniform_(self.fc1.weight)
            nn.init.xavier_uniform_(self.fc2.weight)
            nn.init.xavier_uniform_(self.fc3.weight)
        elif activation == 'relu':
            nn.init.kaiming_uniform_(self.fc1.weight, mode='fan_in', nonlinearity='relu')
            nn.init.kaiming_uniform_(self.fc2.weight, mode='fan_in', nonlinearity='relu')
            nn.init.kaiming_uniform_(self.fc3.weight, mode='fan_in', nonlinearity='relu')

    def forward(self, x):
        x = F.tanh(self.fc1(x)) if self.activation == 'tanh' else F.relu(self.fc1(x))
        x = F.tanh(self.fc2(x)) if self.activation == 'tanh' else F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))  # Use sigmoid for the output layer
        return x

# Load data
train_file_path = "C:\\Rushiil\\ML\\bank-note-1\\bank-note\\train.csv"
test_file_path = "C:\\Rushiil\\ML\\bank-note-1\\bank-note\\test.csv"

X, y = load_data(train_file_path)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

# Specify hyperparameters
depths = [3, 5, 9]
widths = [5, 10, 25, 50, 100]
activation_functions = ['tanh', 'relu']
learning_rate = 1e-3
epochs = 100

# Train and evaluate the neural network for each combination
for activation in activation_functions:
    for depth in depths:
        for width in widths:
            # Define and initialize the model
            model = NeuralNetwork(input_size=X_train.shape[1], hidden_size=width, output_size=1, activation=activation)
            
            # Define loss and optimizer
            criterion = nn.MSELoss()
            optimizer = optim.Adam(model.parameters(), lr=learning_rate)
            
            # Training loop
            for epoch in range(epochs):
                # Forward pass
                outputs = model(X_train_tensor)
                loss = criterion(outputs, y_train_tensor)
                
                # Backward pass and optimization
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
            # Evaluate on validation set
            with torch.no_grad():
                val_predictions = model(X_val_tensor)
                val_loss = criterion(val_predictions, y_val_tensor)

            print(f"Activation: {activation}, Depth: {depth}, Width: {width}, Validation Loss: {val_loss.item()}")


Activation: tanh, Depth: 3, Width: 5, Validation Loss: 0.4061916768550873
Activation: tanh, Depth: 3, Width: 10, Validation Loss: 0.0804738700389862
Activation: tanh, Depth: 3, Width: 25, Validation Loss: 0.023210644721984863
Activation: tanh, Depth: 3, Width: 50, Validation Loss: 0.007043940480798483
Activation: tanh, Depth: 3, Width: 100, Validation Loss: 0.00419103167951107
Activation: tanh, Depth: 5, Width: 5, Validation Loss: 0.18924358487129211
Activation: tanh, Depth: 5, Width: 10, Validation Loss: 0.05489877611398697
Activation: tanh, Depth: 5, Width: 25, Validation Loss: 0.014229507185518742
Activation: tanh, Depth: 5, Width: 50, Validation Loss: 0.008316590450704098
Activation: tanh, Depth: 5, Width: 100, Validation Loss: 0.0037674817722290754
Activation: tanh, Depth: 9, Width: 5, Validation Loss: 0.2332855463027954
Activation: tanh, Depth: 9, Width: 10, Validation Loss: 0.06135578081011772
Activation: tanh, Depth: 9, Width: 25, Validation Loss: 0.018061218783259392
Activatio